In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Initialization of Variables
P_thruster = 1250000 #newtons
P_lift = 1400000 #newtons
P_weight = 500000 #newtons

alpha = 5 #degrees
alpha = alpha * np.pi/180
vC_12 = 0.3 #Poisson's ratio
vC_21 = EC_22*vC_12/EC_11

# Equations for P1 & P2 (Horizontal and Vertical Resultant Forces)
P1 = P_lift*np.sin(alpha) - P_thruster
P2 = P_lift*np.cos(alpha) - P_weight*np.cos(alpha)

# Axial Displacement
x1 = np.linspace(0,20,50)
S = 1
u1 = (P_lift*np.sin(alpha)*x1 - P_thruster*x1)/S

# Axial Strain
eps1 = (P_lift*np.sin(alpha) - P_thruster)/S

def getT(beta):
    beta = beta * np.pi/180
    T = np.array([[np.cos(beta)**2, np.sin(beta)**2, 2*np.sin(beta)*np.cos(beta)],
                 [np.sin(beta)**2, np.cos(beta)**2,-2*np.sin(beta)*np.cos(beta)],
                 [-1*np.sin(beta)*np.cos(beta), np.sin(beta)*np.cos(beta),np.cos(beta)**2 - np.sin(beta)**2]])
    return T

Q_bar = np.invert()
